# Problem 14.12 Work and Temperature Change Upon Adiabatic Compression of Oxygen

A stream of oxygen is compressed by a compressor from a pressure P1  = 1 bar to P2 = 10 bar. The flow rate of the oxygen stream is 250 kg/h and the temperature is 25°C.

What is the power of the compressor, and the outlet temperature of the gas?

## Solution

This is a series of PH, PS and PT flashes.

In [1]:
from scipy.constants import bar, hour
from thermo import ChemicalConstantsPackage, SRKMIX, IGMIX, CEOSGas, CEOSLiquid, FlashPureVLS
fluid = 'oxygen'
constants, correlations = ChemicalConstantsPackage.from_IDs([fluid])

T1 = 25 + 273.15
P1 = 1*bar
P2 = 10*bar
zs = [1]
eta_isentropic = 0.75
eta_mechanical = 0.95

In [2]:
eos_kwargs = dict(Tcs=constants.Tcs, Pcs=constants.Pcs, omegas=constants.omegas)
liquid = CEOSLiquid(SRKMIX, HeatCapacityGases=correlations.HeatCapacityGases, eos_kwargs=eos_kwargs)
gas = CEOSGas(SRKMIX, HeatCapacityGases=correlations.HeatCapacityGases, eos_kwargs=eos_kwargs)
SRK_flasher = FlashPureVLS(constants, correlations, liquids=[liquid], gas=gas, solids=[])

gas = CEOSGas(IGMIX, HeatCapacityGases=correlations.HeatCapacityGases, eos_kwargs=eos_kwargs)
ideal_flasher = FlashPureVLS(constants, correlations, gas=gas, liquids=[], solids=[])


In [3]:
# Flash at inlet conditions to obtain initial enthalpy
state_1 = SRK_flasher.flash(T=T1, P=P1)
# Flash at outlet condition - entropy is conserved by compressors and expanders!
state_2_ideal = SRK_flasher.flash(S=state_1.S(), P=P2)
# Compute the change in enthalpy
delta_H_ideal = (state_2_ideal.H()-state_1.H())
# The definition of isentropic efficiency means that the actual amount of heat added is
# dH_actual = dH_idea/eta_isentropic
H_added_to_fluid_actual = delta_H_ideal/eta_isentropic

state_2 = SRK_flasher.flash(H=state_1.H() + H_added_to_fluid_actual, P=P2)

# To compute the actual power, itis more convinient to use the mass enthalpy
actual_power_per_kg = (state_2.H_mass() - state_1.H_mass())/(eta_mechanical) # W/kg
actual_power = actual_power_per_kg*250/hour
print('With the SRK EOS:')
print(f'The actual power is {actual_power:.0f} W')
print(f'The actual outlet temperature is {state_2.T: .2f} K')

With the SRK EOS:
The actual power is 24368 W
The actual outlet temperature is  643.85 K


In [4]:
# Flash at inlet conditions to obtain initial enthalpy
state_1 = ideal_flasher.flash(T=T1, P=P1)
# Flash at outlet condition - entropy is conserved by compressors and expanders!
state_2_ideal = ideal_flasher.flash(S=state_1.S(), P=P2)
# Compute the change in enthalpy
delta_H_ideal = (state_2_ideal.H()-state_1.H())
# The definition of isentropic efficiency means that the actual amount of heat added is
# dH_actual = dH_idea/eta_isentropic
H_added_to_fluid_actual = delta_H_ideal/eta_isentropic

state_2 = ideal_flasher.flash(H=state_1.H() + H_added_to_fluid_actual, P=P2)

# To compute the actual power, itis more convinient to use the mass enthalpy
actual_power_per_kg = (state_2.H_mass() - state_1.H_mass())/(eta_mechanical) # W/kg
actual_power = actual_power_per_kg*250/hour
print('With the ideal EOS:')
print(f'The actual power is {actual_power:.0f} W')
print(f'The actual outlet temperature is {state_2.T: .2f} K')

With the ideal EOS:
The actual power is 24341 W
The actual outlet temperature is  643.68 K
